In [2]:
import tensorflow as tf
import numpy as np

from tensorflow import keras
from tensorflow.keras import Input, layers
from tensorflow.keras.models import Sequential, Model

seq_model = Sequential() # 익숙한 Sequential 모델
seq_model.add(layers.Dense(32, activation = 'relu', input_shape = (64,)))
seq_model.add(layers.Dense(32, activation = 'relu'))
seq_model.add(layers.Dense(10, activation = 'softmax'))

input_tensor = Input(shape = (64,)) # 함수형 API로 만든 모델
x = layers.Dense(32, activation = 'relu')(input_tensor) # 함수처럼 사용하기 위해 층 객체를 만든다.
x = layers.Dense(32, activation = 'relu')(x)
output_tensor = layers.Dense(10, activation = 'softmax')(x) # 텐서와 함께 층을 호출하면 텐서를 반환

# 파이썬에는 클래스 객체를 함수처럼 호출할 수 있는 특수한 __call__() 메서드가 있다. 케라스는 이 메서드를 즐겨 사용한다.
# dense(input_tensor)를 다르게 쓰면 dense.__call__(input_tensor)와 같다.
# 어떤 객체가 함수 호출이 가능한지 알려면 callable() 내장 함수의 반환값을 확인

model = Model(input_tensor, output_tensor) # 입력과 출력 텐서를 지정하여 모든 층을 추출해 Model클래스의 객체를 만든다.

seq_model.summary()

model.summary()

'''
# Model 객체를 사용한 컴파일, 훈련 평가 API는 Sequential 클래스와 같다.

model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy')
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))
model.fit(x_train, y_train, epochs = 10, batch_size = 128)
score = model.evaluate(x_train, y_train)
'''

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_7 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_8 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_9 (Dense)              (None, 32)                2080      
___________________________________

In [6]:
# 2개의 입력을 가진 질문-응답 모델의 함수형 API 구현하기
# 질문-응답 모델, 2개의 입력(자연어 질문, 답변에 필요한 정보가 담겨 있는 텍스트), 1개의 출력(답)

import tensorflow as tf
import numpy as np


from tensorflow import keras
from tensorflow.keras import layers, Input
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape = (None,), dtype = 'int32', name = 'text') 
    # 텍스트 입력은 길이가 정해지지 않은 정수 시퀀스이다. 입력 이름을 지정할 수 있다.
embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input) # 입력을 크기가 64인 벡터의 시퀀스로 임베딩
encoded_text = layers.LSTM(32)(embedded_text) # LSTM을 사용하여 이 벡터를 하나의 벡터로 인코딩

question_input = Input(shape = (None,), dtype = 'int32', name = 'question') # 질문도 동일한 과정을 거친다.(층 객체는 다르다.)
embedded_question = layers.Embedding(question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question], axis = -1)
    # concatenated = layers.Concatenate(axis = -1)([encoded_text, encoded_question])과 동일, (axis = -1) 이 기본값
    # 인코딩된 질문과 텍스트를 연결
    
answer = layers.Dense(answer_vocabulary_size, activation = 'softmax')(concatenated)

model = Model([text_input, question_input], answer) # 모델 객체를 만들고 2개의 입력과 출력을 주입
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.summary()


# 다중 입력 모델에 데이터 주입하기

    # 입력인 다중인 모델을 훈련 할 떄는 넘파이 배열의 리스트를 주입하거나 입력 이름과 넘파이 배열로 이루어진 딕셔너리 모델의 입력으로 주입.
    
num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size, size = (num_samples, max_length))
    # numpy.random.randint(low, high=None, size=None)
question = np.random.randint(1, question_vocabulary_size, size = (num_samples, max_length))

answers = np.random.randint(0, answer_vocabulary_size, size = num_samples)
answers = to_categorical(answers) # 정답이 정수가 아닌 원-핫-인코딩된 벡터

model.fit([text, question], answers, epochs = 10, batch_size = 128) # 리스트 입력을 사용하여 학습
model.fit({'text' : text, 'question' : question}, answers, epochs = 10, batch_size = 128) # 딕셔너리 입력 사용(이름 지정)

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None, None)]       0                                            
__________________________________________________________________________________________________
question (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 64)     640000      text[0][0]                       
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 32)     320000      question[0][0]                   
____________________________________________________________________________________________

In [10]:
# 3개의 출력을 가진 함수형 API 구현하기
# 소셜 미디어에서 익명 사용자의 포스트를 입력(1D ConvNet)으로 받아 그 사람의 나이, 성별, 소득 수준 등을 예측
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras.models import Model

vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape = (None,), dtype = 'int32', name = 'posts')
embedded_posts = layers.Embedding(vocabulary_size, 256)(posts_input)
x = layers.Conv1D(128, 5, activation = 'relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation = 'relu')(x)
x = layers.Conv1D(256, 5, activation = 'relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation = 'relu')(x)
x = layers.Conv1D(256, 5, activation = 'relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation = 'relu')(x)

age_prediction = layers.Dense(1, name = 'age')(x)
income_prediction = layers.Dense(num_income_groups, activation = 'softmax', name = 'income')(x)
gender_prediction = layers.Dense(1, activation = 'sigmoid', name = 'gender')(x)

model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])



# 다중 출력 모델의 컴파일 옵션 : 다중 손실
model.compile(optimizer = 'rmsprop', loss = ['mse', 'categorical_crossentropy', 'binary_crossentropy'])
    # 혹은
model.compile(optimizer = 'rmsprop',
             loss = {'age':'mse', 'income':'categorical_crossentropy', 'gender':'binary_crossentropy'})



# 다중 출력 모델의 컴파일 옵션 : 손실 가중치 (손실 값이 불균형 하거나 스케일이 다를 때, 가중치로 조정)
model.compile(optimizer = 'rmsprop',
              loss = ['mse', 'categorical_crossentropy', 'binary_crossentropy'],
              loss_weights = [0.25, 1., 10.])
    # 혹은
model.compile(optimizer = 'rmsprop',
              loss = {'age':'mse', 'income':'categorical_crossentropy', 'gender':'binary_crossentropy'},
              loss_weights = {'age':0.25, 'income':1., 'gender':10.})



# 다중 출력 모델에 데이터 주입
model.fit(posts, [age_target, income_targets, gender_targets], epochs = 10, batch_size = 64) # 넘파이 배열 일 때
    #혹은
model.fit(posts,
          {'age':age_targets, 'income':income_targets, 'gender':gender_targets},
          epochs = 10,
          batch_size = 64) # 딕셔너리 모델



TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'

In [ ]:
# Inception V3 모듈 구현

import tensorflow as tf
 
from tensorflow 